# Clustering of the graph

Before doing optimisation we are splitting the graph into separate subclusters.

In [ ]:
import sys
import networkx as nx
from ortools.sat.python import cp_model

from scripts import graph_osm_loader, utils, clustering, centroids_graph_builder

sys.path.append('../')

GRAPH_ID = 'R13470549'  # R13470549 R2555133 R3766483
# graphs from library of graphs graph_osm_loader.py

g = graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))


# Clustering algorithm

Here we will need to select point as representative of each centroid subcluster.

This algorithm is using libraries from https://github.com/sb-ai-lab/Ride/fork


## Data loading

Here we make the example of loading data from both
GRAPH_ID = 'R13470549'  and as well as from the osmnx.

In [ ]:
# Import required libraries
import sys
import networkx as nx
#from ortools.sat.python import cp_model
import community  # Direct import (works if package is installed correctly)
# Import custom modules from scripts directory
from scripts import graph_osm_loader, utils, clustering, centroids_graph_builder

# Add parent directory to Python path for module resolution
sys.path.append('../')

# Define graph ID and load OpenStreetMap graph
GRAPH_ID = 'R13470549'  # Example graph IDs: R13470549 R2555133 R3766483
g = graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))  # Print graph statistics

partition = community.best_partition(G)

# Initialize community detection resolver using Louvain + KMeans clustering
cms_resolver = clustering.LouvainKMeansCommunityResolver(resolution=1)

# Build centroid graph with timing information
t, cg = centroids_graph_builder.CentroidGraphBuilder().build_with_time(g, cms_resolver)

# Generate sample points for selected problem
N = 10
points = list({p for p, u in utils.read_points(GRAPH_ID, g, num=N)})
points


In [ ]:

# Precompute all-pairs shortest paths between sample points
dst = {(u, v): nx.dijkstra_path_length(g, u, v, weight='length') for u in points for v in points}

def get_model(points, dst_matrix, START=None, cms_order=None, initial_X=None, initial_U=None):
    """Here it will be related to the Traveling Salesman Problem (TSP) with optional constraints"""
    if START is None:
        START = points[0]

    # Initialize decision variables
    X = {}  # X[u,v] = 1 if arc u->v is selected
    U = {}  # U[u] = position in tour (for MTZ constraints)
    model = cp_model.CpModel()

    # Create boolean variables for each possible arc
    for v in points:
        for u in points:
            X[v, u] = model.new_bool_var(f'r_{v}_{u}')
        model.add(X[v, v] == 0)  # No self-loops
        U[v] = model.new_int_var(name=f'u_{v}', lb=0, ub=N-1)  # Position variables

    # Flow conservation constraints
    for u in points:
        model.add(sum(X[u, v] for v in points) == sum(X[v, u] for v in points))

    # Exactly one incoming edge for each node (except start)
    for u in points:
        if u == START:
            pass
        else:
            model.add(sum(X[v, u] for v in points) == 1)

    # Exactly one outgoing edge from start node
    model.add(sum(X[START, v] for v in points if v != START) == 1)
    U[START] = 0  # Start node position is 0

    # MTZ subtour elimination constraints
    for u in points:
        for v in points:
            if v != START:
                model.add(U[u] + 1 <= U[v] + (1 - X[u, v]) * 1000)

    # Optional cluster ordering constraints
    if cms_order is not None:
        vals = list(cms_order.items())
        vals.sort(key=lambda x: x[1])
        for i in range(len(vals)-2):
            (k1, v1) = vals[i]
            (k2, v2) = vals[i+2]
            if v1 == 0 or v1 == len(vals) - 1:
                continue
            set1 = set()
            set2 = set()
            # Create sets of nodes for adjacent clusters
            for u in points:
                if g.nodes()[u]['cluster'] == k2:
                    set2.add(u)
                elif g.nodes()[u]['cluster'] == k1:
                    set1.add(u)
            # Add ordering constraints between clusters
            for u in set1:
                for v in set2:
                    model.add(U[u] <= U[v])

    # Add initial solution hints if provided
    if initial_X is not None:
        for (u,v),val in initial_X.items():
            model.add_hint(X[u,v],val)
    if initial_U is not None:
        for u,val in initial_U.items():
            model.add_hint(U[u],val)

    # Objective: Minimize total tour distance
    obj = sum(X[a, b] * int(dst_matrix[a, b]) for a in points for b in points)
    model.minimize(obj)
    return model, X, U

# Get cluster information for points
cms_points = list({g.nodes()[p]['cluster'] for p in points})
# Precompute distances between cluster centroids
cms_dst = {(u, v): nx.dijkstra_path_length(cg.g, u, v, weight='length') for u in cms_points for v in cms_points}
